# Basic examples

In this notebook we will show main **mpytools** utilities. You need to have installed **mpytools** with:

python -m pip install git+https://github.com/cosmodesi/mpytools

Everything presented here is MPI'ed, i.e. memory and computation can be split among multiple processes
(e.g. ``mpiexec -np 10 python yourscript.py`` or ``srun -n 10 yourscript.py``).

In [1]:
import os

import numpy as np

import mpytools as mpy
from mpytools import Catalog

## Array
Let's create some MPI-scattered array:

In [2]:
local_array = np.ones((10, 2), dtype='f8')
array = mpy.array(local_array)  # local_array on each process
# One can also start with a global array:
mpicomm = mpy.COMM_WORLD
global_array = np.ones((100, 2), dtype='f8') if mpicomm.rank == 0 else None
array = mpy.array(global_array, mpicomm=mpicomm, mpiroot=0)
# Alternatively, one can directly create an empty array,
# full of 0, 1, any value using empty, zeros, ones, full, respectively; e.g.:
zeros = mpy.zeros((10, 2))  # array of shape (10, 2) on each rank
# Alternatively, one can provide the global shape instead:
empty = mpy.empty(cshape=(100, 2))
assert empty.cshape == (100, 2)
assert empty.csize == 200  # csize is the collective array size; to get the local array size: empty.size

### Concatenating

Arrays can be concatenated locally, or collectively:

In [3]:
# Local concatenation, e.g. on first rank we will have the beginning of all three input arrays
# this is not costly as no data is exchanged between various ranks
test = np.concatenate([zeros, empty])
# If one wants to preserve the global order, one should rather do (notice the starting 'c' meaning 'collective'):
test = mpy.cconcatenate([zeros, empty])
# This requires data to be passed between all processes

### Slicing
Arrays can be sliced locally, or collectively:

In [4]:
# Local slice, i.e. impacts each process independently
test2 = test[2:80]  # or test[slice(2, 80)]
# Collective slice, i.e. slice the array globally (including load balancing on all processes)
test2 = test.cslice(60)
assert test2.cshape[0] == 60

### Miscellaneous
Note that *any* numpy function will apply to mpy.array. Main collective operations are also implemented (csum, cstd, etc.)

In [5]:
# One can apply any numpy function (locally) to a mpyarray
mean = np.mean(array)  # local mean
cmean = mpy.cmean(array)  # collective mean
# As in numpy, a few functions are attached as methods to mpy.array, e.g.:
mean = array.mean()  # local mean
cmean = array.cmean()  # collective mean
# Same for csum, cvar, cstd, cmin, cmax...

## Catalog
Let's create some catalog:

In [6]:
size = 1000
rng = mpy.random.MPIRandomState(size, seed=42)  # invariant under number of processes
catalog = Catalog(data={'RA': rng.uniform(0., 20.), 'DEC': rng.uniform(-10., 10.),
                        'Z': rng.uniform(1., 2.), 'Position': rng.uniform(0., 1., itemshape=3)})

### Column access

In [7]:
# Columns can be accessed locally through:
ra = catalog['RA']  # or catalog.get('RA')
# This returns a mpy.array, so one can do:
ra.cmean()
# To get a standard numpy array one can do:
ra = catalog.get('RA', return_type='nparray')
# The full column (concatenated across all ranks) can be accessed through:
ra = catalog.cget('RA')
# Alternatively, ra = catalog['RA'].gather()

### I/Os
Catalogs can be written/read from many formats: 'fits', 'hdf5', 'npy', 'bigfile', 'asdf'.

In [8]:
tmp_dir = '_tests'

fn = os.path.join(tmp_dir, 'tmp.fits')  # same for hdf5, npy, bigfile, asdf
catalog.write(fn)
test = Catalog.read(fn)  # you can provide optional arguments, e.g. "ext" for fits --- see mpytools.io.FitsFile
assert test == catalog  # check catalog equality

fns = [os.path.join(tmp_dir, 'tmp1.fits'), os.path.join(tmp_dir, 'tmp2.fits')]
test.write(fns)  # one can split in many catalogs
test = Catalog.read(fns)  # read multiple catalogs at the same time
# Note that .read() only reads file headers, so is almost free
# columns are only read when accessing them, e.g.:
test['RA'][:10]  # this is a mpyarray, so one can use all methods mentioned above
print('Catalog columns are', test.columns())
print('Catalog columns currently read are', test.data.keys())

# One can even write to / read from different file types (using default options for each file type)
fns = [os.path.join(tmp_dir, 'tmp1.fits'), os.path.join(tmp_dir, 'tmp2.hdf5')]
test.write(fns)
test = Catalog.read(fns)

Catalog columns are ['RA', 'DEC', 'Z', 'Position']
Catalog columns currently read are dict_keys(['RA'])


### Concatenating
Catalogs can be concatenated locally, or collectively:

In [9]:
# Local concatenation, e.g. on first rank we will have the beginning of all three input catalogs
# this is not costly as no data is exchanged between various ranks
# Again, columns that are not read yet will be concatenated when accessing them.
test3 = Catalog.concatenate(test, test, test)
# If one wants to preserve the global order, one should rather do (notice the starting 'c' meaning 'collective'):
test3 = Catalog.cconcatenate(test, test, test)
# This requires data to be passed between all processes

### Slicing
Catalogs can be sliced locally, or collectively:

In [10]:
# Local slice, i.e. impacts each process independently
test2 = test[2:80]  # or test.slice(2, 80), or test[np.arange(2, 80)]
# Collective slice, i.e. resize the catalog globally (including load balancing on all processes)
test2 = test.cslice(60)
assert test2.csize == 60  # csize is the collective catalog size; to get the local catalog size: test2.size
# Internally, this will slice column 'RA' (which has already been read above)
# but has virtually no cost for columns that have not been read yet:
# slicing will be applied when accessing them, e.g.:
assert test2['Position'].cshape[0] == 60

### Miscellaneous
Other helpful routines:

In [11]:
# Get (collective) indices
cindices = catalog.cindices()
assert np.all(cindices.gather(mpiroot=None) == np.arange(catalog.csize))
# Get column of zeros of shape (catalog.size, 3)
catalog['zeros'] = catalog.zeros(itemshape=3)
# Same for empty, ones, falses, trues, nans, full
catalog['trues'] = catalog.trues()
print('Catalog columns are', catalog.columns())
print('Catalog columns starting with "t" are', catalog.columns(include='t*'))
del catalog['trues']
print('Catalog columns, except "RA" now are', catalog.columns(exclude='RA'))

Catalog columns are ['RA', 'DEC', 'Z', 'Position', 'zeros', 'trues']
Catalog columns starting with "t" are ['trues']
Catalog columns, except "RA" now are ['DEC', 'Z', 'Position', 'zeros']
